In [1]:
import pyspark

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local") \
    .appName("Spark") \
    .config('spark.sql.repl.eagerEval.enabled', 'True') \
    .getOrCreate()

In [ ]:
#spark.conf.set("spark.executor.memory", '4g')
#spark.conf.set('spark.executor.cores', '3')
#spark.conf.set('spark.cores.max', '3')
#spark.conf.set("spark.driver.memory",'4g')

In [3]:
sc = spark.sparkContext

In [4]:
sc.version

'3.1.1'

In [5]:
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.sql.warehouse.dir', 'file:/home/jovyan/code/spark-warehouse'),
 ('spark.app.name', 'Spark'),
 ('spark.sql.repl.eagerEval.enabled', 'True'),
 ('spark.app.id', 'local-1620297360260'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', 'b5a3f48f8f4b'),
 ('spark.driver.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.startTime', '1620297359181'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.submit.pyFiles', ''),
 ('spark.submit.deployMode', 'client'),
 ('spark.executor.extraJavaOptions',
  '-Dio.netty.tryReflectionSetAccessible=true'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.port', '33293')]

In [4]:
TcgaTargetGTEX_phenotype = spark.read.options(header=True, inferSchema=True, delimiter='\t').csv("../data/TcgaTargetGTEX_phenotype.txt.gz")

In [5]:
TcgaTargetGTEX_phenotype.createOrReplaceTempView("phenotype")

In [7]:
GTEx_breast = spark.sql("SELECT * FROM phenotype where _study = 'GTEX' and _primary_site = 'Breast'")

In [9]:
type(GTEx_breast)

pyspark.sql.dataframe.DataFrame

In [11]:
GTEx_breast.head()

Row(sample='GTEX-T6MN-0726-SM-32PML', detailed_category='Breast - Mammary Tissue', primary disease or tissue='Breast - Mammary Tissue', _primary_site='Breast', _sample_type='Normal Tissue', _gender='Male', _study='GTEX')

In [47]:
GTEx_breast_sample = GTEx_breast.select("sample").rdd.flatMap(lambda x: x).collect()

In [6]:
Ensembl_gene_type = spark.read.options(header=True, inferSchema=True, delimiter=',').csv("../data/Ensembl_gene_name_type.csv")

In [7]:
genetype = Ensembl_gene_type.createOrReplaceTempView("genetype")

In [9]:
TcgaTargetGtex_RSEM_Hugo_norm_count = spark.read.options(header=True, inferSchema=True, delimiter='\t').csv("../data/TcgaTargetGtex_RSEM_Hugo_norm_count.gz")

In [12]:
genelist = TcgaTargetGtex_RSEM_Hugo_norm_count.select("sample").rdd.flatMap(lambda x: x).collect()

In [15]:
genes = TcgaTargetGtex_RSEM_Hugo_norm_count.select("sample")

In [17]:
gene = genes.createOrReplaceTempView("gene")

In [20]:
Ensembl_TCGA_GETx_gene_type = spark.sql("SELECT Gene_name, Gene_type FROM gene INNER JOIN genetype ON gene.sample = genetype.gene_name")

In [22]:
Ensembl_TCGA_GETx_gene_type.write.format("csv").save("../data/Ensembl_TCGA_GETx_gene_type.csv")

In [25]:
Ensembl_TCGA_GETx_gene_type.groupby("Gene_type").count()

Gene_type,count
protein_coding,21721
miRNA,2743
lincRNA,6589


In [68]:
type(GTEx_breast_sample)

list

In [69]:
test = ['sample']

In [70]:
type(test)

list

In [71]:
test.extend(GTEx_breast_sample)

In [74]:
GTEx_Breast_norm_count = TcgaTargetGtex_RSEM_Hugo_norm_count.select(test)

In [104]:
pandasDF = GTEx_Breast_norm_count.toPandas().set_index('sample').transpose()

In [121]:
pandasDF.head()

sample,CTD-2588J6.1,RP11-433M22.1,CTD-2588J6.2,RP3-415N12.1,RP11-16E12.2,RP11-433M22.2,PSMA2P3,RP11-181G12.4,RP11-181G12.2,AC007272.3,...,RP11-390F4.3,GNGT2,GNGT1,SELT,DIAPH2-AS1,SELV,NFIX,SELP,AC002386.1,RP11-526P5.2
GTEX-T6MN-0726-SM-32PML,1.4482,0.0,0.0,2.8177,3.5074,0.8986,0.0000,0.8986,6.8026,0.0000,...,6.4923,3.6131,0.0000,11.0095,1.4482,1.4482,13.2609,9.8067,0.0,0.0
GTEX-13NZB-2126-SM-5MR4Y,0.0000,0.0,0.0,0.0000,1.0154,0.0000,0.0000,0.0000,6.6302,0.0000,...,5.8881,4.7844,0.0000,11.0757,1.0154,3.1972,12.6347,9.8917,0.0,0.0
GTEX-ZU9S-1926-SM-5NQBP,0.0000,0.0,0.0,1.4965,1.9000,0.0000,0.0000,0.0000,6.7367,0.0000,...,6.3745,4.5215,0.0000,11.4081,2.5108,3.9609,13.2720,8.6533,0.0,0.0
GTEX-11DXW-0626-SM-5N9ER,0.0000,0.0,0.0,1.0942,2.4699,0.0000,0.0000,1.0942,6.5205,0.0000,...,5.7641,4.7636,0.0000,10.8074,1.0942,1.7093,12.8925,11.0399,0.0,0.0
GTEX-UPK5-2326-SM-3P5Z8,1.2686,0.0,0.0,1.9330,2.7305,0.0000,1.2686,0.0000,6.5973,1.8979,...,5.6328,5.2937,0.7694,10.9437,2.5686,1.2686,12.6923,11.0987,0.0,0.0


In [103]:
spark.conf.set("spark.sql.execution.arrow.enabled","true")

In [ ]:
sparkDF=spark.createDataFrame(pandasDF.reset_index()) 

In [118]:
sparkDF.columns[0:10]

['CTD-2588J6.1',
 'RP11-433M22.1',
 'CTD-2588J6.2',
 'RP3-415N12.1',
 'RP11-16E12.2',
 'RP11-433M22.2',
 'PSMA2P3',
 'RP11-181G12.4',
 'RP11-181G12.2',
 'AC007272.3']

In [120]:
import time

start = time.time()
sparkDF.corr('CTD-2588J6.1', 'RP11-433M22.1')
print("time:", time.time() - start)

Py4JJavaError: An error occurred while calling o1098.stat.
: java.lang.OutOfMemoryError: Java heap space


In [ ]:
sparkDF.printSchema()

In [ ]:
sparkDF.show()

https://stackoverflow.com/questions/60430645/get-a-correlation-matrix-in-pyspark-with-a-large-dataset

In [ ]:
df = GTEx_Breast_norm_count.select(GTEx_breast_sample)

In [ ]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

# convert to vector column first
vector_col = "corr_features"
assembler = VectorAssembler(inputCols=df.columns, outputCol=vector_col)
df_vector = assembler.transform(df).select(vector_col)

# get correlation matrix
matrix = Correlation.corr(df_vector, vector_col)

In [83]:
matrix

pearson(corr_features)
1.0 ...


In [96]:
corr = matrix.collect()[0]["pearson({})".format(vector_col)].values